## Topic Modeling
> Module with methods to perform topic modeling

In [ ]:
#|default_exp modeling

In [ ]:
#| export
from sklearn.datasets import fetch_20newsgroups
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
import json

import re
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import pandas as pd

In [ ]:
#| export
from openai import OpenAI
from bertopic.backend import OpenAIBackend

import tiktoken
from bertopic import BERTopic

In [ ]:
#| export
def load_data_from_json(filename):
    """Load data from a JSON file."""
    with open(filename, 'r', encoding='utf-8') as file:
        return json.load(file)

In [ ]:
#| export
def remove_stop_words(issue_body):
    """
    Clean the text of a GitHub issue for topic modeling.
    """
    # Remove URLs
    issue_body = re.sub(r'http\S+', ' ', issue_body)

    # Remove user mentions
    issue_body = re.sub(r'@\S+', ' ', issue_body)

    # Remove special characters and numbers
    issue_body = re.sub(r'[^A-Za-z\s]', ' ', issue_body)

    # Tokenize text
    tokens = word_tokenize(issue_body)

    # Convert to lower case
    tokens = [word.lower() for word in tokens]

    # Remove punctuation
    tokens = [word for word in tokens if word not in string.punctuation]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Optional: Lemmatization/Stemming (can be added based on requirement)

    # Rejoin tokens into a string
    cleaned_text = ' '.join(tokens)

    return cleaned_text

In [ ]:
#| export
def deep_clean_text(text):
    """
    Clean and curate the text from a GitHub issue for better processing with BERTopic.

    Parameters:
    issue_body (str): The text content of a GitHub issue.

    Returns:
    str: Cleaned and curated text.
    """

    # Remove HTML tags using BeautifulSoup
    issue_body = BeautifulSoup(text, "html.parser").get_text()

    # Remove URLs
    issue_body = re.sub(r'http\S+', ' ', issue_body)

    # Remove GitHub flavored markdown
    issue_body = re.sub(r'```[a-z]*\n[\s\S]*?\n```', ' ', issue_body)  # Code blocks
    issue_body = re.sub(r'`[^`]*`', ' ', issue_body)  # Inline code
    issue_body = re.sub(r'\*\*[^*]*\*\*', ' ', issue_body)  # Bold text
    issue_body = re.sub(r'\*[^*]*\*', ' ', issue_body)  # Italic text
    issue_body = re.sub(r'~~[^~]*~~', ' ', issue_body)  # Strikethrough
    issue_body = re.sub(r'\[[^\]]*\]\([^\)]*\)', ' ', issue_body)  # Hyperlinks

    return remove_stop_words(issue_body)

In [ ]:
#| export
def get_body_from_issues(issues: list):
  return [remove_stop_words(issue['body']) for issue in issues if issue['body'] is not None]

In [ ]:
#| export
def generate_topic_name(keywords, OPEN_AI_KEY):
    #openai.api_key = OPEN_AI_KEY
    client = OpenAI(api_key=OPEN_AI_KEY)
    response = client.chat.completions.create(model="gpt-4",
    messages=[{"role": "user", "content": f"Generate a short descriptive name for the category of a topic, based on these keywords: {', '.join(keywords)}"}])
    print(response.choices[0].message.content)
    return response.choices[0].message.content